In [ ]:
import os
import json
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, clear_output
from dotenv import load_dotenv

# === Setup ===
load_dotenv()
client = OpenAI(api_key='')

# Directory for saving conversations
SAVE_DIR = "./tmp"
os.makedirs(SAVE_DIR, exist_ok=True)

# Load all saved conversations
def load_all_conversations():
    convos = {}
    for filename in os.listdir(SAVE_DIR):
        if filename.endswith(".json"):
            path = os.path.join(SAVE_DIR, filename)
            with open(path, "r", encoding="utf-8") as f:
                convo_name = filename.replace(".json", "")
                convos[convo_name] = json.load(f)
    return convos

# Save a conversation to disk
def save_conversation(name, convo):
    path = os.path.join(SAVE_DIR, f"{name}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(convo, f, indent=2)

# Initialize
conversations = load_all_conversations()
if not conversations:
    conversations["convo_0"] = []
current_convo_name = list(conversations.keys())[0]

# === Widgets ===
instructions_box = widgets.Textarea(
    value="You are a helpful assistant.",
    placeholder="Enter instructions for GPT-4o here",
    description="Instructions:",
    layout=widgets.Layout(width='100%', height='80px')
)

prompt_box = widgets.Textarea(
    value="",
    placeholder="Type your message here",
    description="Prompt:",
    layout=widgets.Layout(width='100%', height='80px')
)

output_area = widgets.Output(layout=widgets.Layout(border='1px solid gray', height='300px', overflow='auto'))

send_button = widgets.Button(description="Send", button_style='success')
new_button = widgets.Button(description="New Conversation", button_style='info')
load_dropdown = widgets.Dropdown(options=list(conversations.keys()), description="Load:")

# === GPT-4o call using correct API ===
def ask_gpt4o(instructions: str, user_input: str) -> str:
    response = client.responses.create(
        model="gpt-4o",
        instructions=instructions,
        input=user_input,
    )
    return response.output_text

# === UI Actions ===
def send_message(_):
    global current_convo_name
    user_input = prompt_box.value.strip()
    if not user_input:
        return

    convo = conversations[current_convo_name]
    convo.append({"role": "user", "content": user_input})
    save_conversation(current_convo_name, convo)

    with output_area:
        clear_output()
        print_chat(convo)

        try:
            # Combine history as a single input string
            combined_input = "\n".join(
                f"{'You' if m['role']=='user' else 'Assistant'}: {m['content']}"
                for m in convo
            )
            assistant_reply = ask_gpt4o(instructions_box.value, combined_input)
            convo.append({"role": "assistant", "content": assistant_reply})
            save_conversation(current_convo_name, convo)
            clear_output()
            print_chat(convo)
        except Exception as e:
            print(f"Error: {e}")

    prompt_box.value = ""

def start_new_convo(_):
    global current_convo_name
    idx = len(conversations)
    name = f"convo_{idx}"
    conversations[name] = []
    current_convo_name = name
    save_conversation(name, [])
    load_dropdown.options = list(conversations.keys())
    load_dropdown.value = name
    with output_area:
        clear_output()
        print(f"Started new conversation: {name}")

def load_convo(change):
    global current_convo_name
    current_convo_name = change["new"]
    with output_area:
        clear_output()
        print_chat(conversations[current_convo_name])

def print_chat(convo):
    print(f"--- Chat: {current_convo_name} ---\n")
    for msg in convo:
        role = msg["role"]
        prefix = "🧑 You" if role == "user" else "🤖 GPT-4o"
        print(f"{prefix}: {msg['content']}\n")

# === Wire up UI ===
send_button.on_click(send_message)
new_button.on_click(start_new_convo)
load_dropdown.observe(load_convo, names="value")

# === Display UI ===
ui = widgets.VBox([
    instructions_box,
    widgets.HBox([prompt_box]),
    widgets.HBox([send_button, new_button, load_dropdown]),
    output_area
])

display(ui)

# Show default conversation
with output_area:
    clear_output()
    print_chat(conversations[current_convo_name])